In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import kendalltau
from sklearn.covariance import GraphicalLasso
from tqdm import tqdm
import datetime

# Step 1: Download S&P 500 data from Yahoo Finance
def download_sp500_data(start_date, end_date):
    sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_table = pd.read_html(sp500_url)
    sp500_df = sp500_table[0]
    sp500_tickers = set(sp500_df['Symbol'].tolist()) - set(['ABNB', 'CEG', 'OTIS', 'GEHC', 'GEV', 'CARR', 'VLTO', 'KVUE', 'SOLV', 'BRK.B', 'BF.B'])
    sp500_tickers = list(sp500_tickers)
    data = yf.download(sp500_tickers, start=start_date, end=end_date)['Adj Close']
    return data

# Step 2: Calculate Kendall's tau correlation matrix
def kendalls_tau_matrix(returns):
    n = returns.shape[1]
    tau_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            tau, _ = kendalltau(returns.iloc[:, i], returns.iloc[:, j])
            tau_matrix[i, j] = tau
            tau_matrix[j, i] = tau
    tau_matrix = np.nan_to_num(tau_matrix)  # Fill NaN values with 0
    return np.sin(np.pi / 2 * tau_matrix)

# Step 3: Smoothly Projected Neighborhood Pursuit (Graphical Lasso)
def graphical_lasso(correlation_matrix, alpha):
    if np.isnan(correlation_matrix).any():
        raise ValueError("Correlation matrix contains NaN values.")
    model = GraphicalLasso(alpha=alpha, max_iter=200)
    model.fit(correlation_matrix)
    return model.precision_

# Step 4: Stability Selection
def stability_selection(returns, alpha, n_subsamples=50, subsample_size_ratio=0.8):
    n_samples, n_stocks = returns.shape
    subsample_size = int(n_samples * subsample_size_ratio)
    
    freq_selected_edges = np.zeros((n_stocks, n_stocks))
    
    for _ in range(n_subsamples):
        subsample = returns.sample(n=subsample_size, replace=False)
        tau_matrix = kendalls_tau_matrix(subsample)
        precision_matrix = graphical_lasso(tau_matrix, alpha)
        non_zero_edges = (precision_matrix != 0).astype(int)
        freq_selected_edges += non_zero_edges
    
    freq_selected_edges /= n_subsamples
    return freq_selected_edges


# Step 5: Independent Node Selection
def select_independent_nodes(freq_selected_edges, threshold=0.05):
    independent_nodes = []
    for i in range(freq_selected_edges.shape[0]):
        if np.sum(freq_selected_edges[i, :]) <= threshold:
            independent_nodes.append(i)
    return independent_nodes

# Main function for backtesting
def backtest_strategy(start_date, end_date, initial_capital=1):
    data = download_sp500_data(start_date, end_date)
    returns = data.pct_change().fillna(0)
    
    lambda_seq = np.logspace(0.1, 1, num=50)
    current_capital = initial_capital
    capital_history = [current_capital]
    
    quarters = pd.date_range(start=start_date, end=end_date, freq='Q')
    
    for i in tqdm(range(1, len(quarters))):
        train_start = quarters[i-1]
        train_end = quarters[i] - pd.Timedelta(days=1)
        test_start = quarters[i]
        test_end = quarters[i+1] - pd.Timedelta(days=1) if i+1 < len(quarters) else end_date
        
        train_returns = returns[train_start:train_end]
        test_returns = returns[test_start:test_end]
        
        if train_returns.empty or test_returns.empty:
            continue
        
        freq_selected_edges = stability_selection(train_returns, lambda_seq)
        independent_nodes = select_independent_nodes(freq_selected_edges)
        
        if not independent_nodes:
            continue
        
        selected_stocks = train_returns.columns[independent_nodes]
        equal_weight = 1 / len(selected_stocks)
        
        quarterly_return = test_returns[selected_stocks].mean(axis=1).sum() * equal_weight
        current_capital *= (1 + quarterly_return)
        capital_history.append(current_capital)
        
    return capital_history

# Run the backtest
start_date = '2005-01-01'
end_date = '2024-07-09'
#capital_history = backtest_strategy(start_date, end_date)

#print(capital_history)


In [ ]:
data = download_sp500_data(start_date, end_date)
returns = data.pct_change().fillna(0)

lambda_seq = np.logspace(-2, 0, num=50)
current_capital = 1
capital_history = [current_capital]

quarters = pd.date_range(start=start_date, end=end_date, freq='Q')

i=1
train_start = quarters[i-1]
train_end = quarters[i] - pd.Timedelta(days=1)
test_start = quarters[i]
test_end = quarters[i+1] - pd.Timedelta(days=1) if i+1 < len(quarters) else end_date

train_returns = returns[train_start:train_end]
test_returns = returns[test_start:test_end]